In [1]:
import warnings
warnings.filterwarnings('ignore')
import copy
import pandas as pd
import numpy as np
# Requests sends and recieves HTTP requests.
import requests

# Beautiful Soup parses HTML documents in python.
from bs4 import BeautifulSoup
import time
import random
import re
from sportsreference.ncaab.boxscore import Boxscore
from sportsreference.ncaab.roster import Player
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
import random
import sportsreference.nba.roster as NBA
from sklearn.ensemble import RandomForestClassifier
import unicodedata

In [2]:
lst1 = []
lst2 = []
count = 0
draft_url = 'https://www.basketball-reference.com/play-index/draft_finder.cgi?request=1&year_min=1985&year_max=2016&round_min=&round_max=&pick_overall_min=&pick_overall_max=&franch_id=&college_id=0&is_active=&is_hof=&pos_is_g=Y&pos_is_gf=Y&pos_is_f=Y&pos_is_fg=Y&pos_is_fc=Y&pos_is_c=Y&pos_is_cf=Y&c1stat=&c1comp=&c1val=&c2stat=&c2comp=&c2val=&c3stat=&c3comp=&c3val=&c4stat=&c4comp=&c4val=&order_by=ws&order_by_asc=&offset=0'
for i in range(45):
    if i >0:
        href_list = soup.find("div", {"class": "p402_premium"}).find('p').find_all('a')
        if len(href_list) == 1:
            count += 1
            print(count)
            if count == 2:
                break
            href = href_list[0].get('href')
        else:
            href = href_list[1].get('href')
        print(href)
        draft_url= 'https://www.basketball-reference.com' + href
        time.sleep(3*random.random())
    r = requests.get(draft_url)
    print(r.status_code)
    soup = BeautifulSoup(r.content, "html")
    div = soup.find("div", {"class": "table_outer_container"})
    table = div.find("tbody")
    rows = table.find_all('tr')
    for row in rows:
        try:
            href = row.find_all('a')[1].get('href')
        except:
            continue
        try:
            nba_id = re.findall('\w+\d{2}', href)[0]
        except:
            continue
        tds = row.find_all('td')
        lst3 = []
        for item in tds:
            lst3.append(item.get_text())
        lst3.append(nba_id)
        lst1.append(lst3)
for i in rows[0].find_all('td'):
    lst2.append(i.get("data-stat"))
        

200
1
/play-index/draft_finder.cgi?request=1&year_min=1985&year_max=2016&round_min=&round_max=&pick_overall_min=&pick_overall_max=&franch_id=&college_id=0&is_active=&is_hof=&pos_is_g=Y&pos_is_gf=Y&pos_is_f=Y&pos_is_fg=Y&pos_is_fc=Y&pos_is_c=Y&pos_is_cf=Y&c1stat=&c1comp=&c1val=&c2stat=&c2comp=&c2val=&c3stat=&c3comp=&c3val=&c4stat=&c4comp=&c4val=&order_by=ws&order_by_asc=&offset=100
200
/play-index/draft_finder.cgi?request=1&year_min=1985&year_max=2016&round_min=&round_max=&pick_overall_min=&pick_overall_max=&franch_id=&college_id=0&is_active=&is_hof=&pos_is_g=Y&pos_is_gf=Y&pos_is_f=Y&pos_is_fg=Y&pos_is_fc=Y&pos_is_c=Y&pos_is_cf=Y&c1stat=&c1comp=&c1val=&c2stat=&c2comp=&c2val=&c3stat=&c3comp=&c3val=&c4stat=&c4comp=&c4val=&order_by=ws&order_by_asc=&offset=200
200
/play-index/draft_finder.cgi?request=1&year_min=1985&year_max=2016&round_min=&round_max=&pick_overall_min=&pick_overall_max=&franch_id=&college_id=0&is_active=&is_hof=&pos_is_g=Y&pos_is_gf=Y&pos_is_f=Y&pos_is_fg=Y&pos_is_fc=Y&pos_

In [3]:
random.shuffle(lst1)
print(len(lst1))

In [4]:
def strip_accents(text):

    try:
        text = unicode(text, 'utf-8')
    except NameError: # unicode is a default on python 3 
        pass

    text = unicodedata.normalize('NFD', text)\
           .encode('ascii', 'ignore')\
           .decode("utf-8")

    return str(text)

In [5]:
def duplicate_college_name(element):
    #make sure last college year is nba draft year
        player = strip_accents(element[5]).replace('.','').replace("'",'').replace(',','')
        split_player = player.split()
        college_id = ('-'.join(split_player) + '-jr-1').lower()
        #print(college_id)
        draft_year = int(element[0][-2:])
        #print(draft_year)
        injuried_year = int(draft_year - 1)
        #print(injuried_year)
        try:
            last_college_year = int(Player(college_id).dataframe.index[-2][0].split('-')[-1])
            #print(last_college_year)
            if ((last_college_year == draft_year) | (last_college_year == injuried_year)):
                return college_id
        except:
            pass
        for attempt in range(1,6):
            split_player = player.split()
            college_id = ('-'.join(split_player) + '-' + str(attempt)).lower()
            try:
                last_college_year = int(Player(college_id).dataframe.index[-2][0].split('-')[-1])
                #print(last_college_year)
                if ((last_college_year == draft_year) | (last_college_year == injuried_year)):
                    return college_id
                continue
            except:
                continue
        return 'not_college_player'

In [6]:
all_star_list = []
url = 'https://www.basketball-reference.com/allstar/NBA-allstar-career-stats.html'
r = requests.get(url)
soup1 = BeautifulSoup(r.content, "html")
div = soup1.find("div", {"class": "table_outer_container"})
table = div.find("tbody")
rows = table.find_all('tr')
for row in rows:
    a_tag =row.find('th').find('a')
    if a_tag == None:
        continue
    player = a_tag.get_text()
    all_star_list.append(player)

In [7]:
def rename_nba_index(nba_player_series):
    NBA_index = nba_player_series.index
    new_index = []
    for index in NBA_index:
        new_index.append(index+'_NBA')
    nba_player_series.index = new_index
    return nba_player_series
def rename_college_index(college_player_series):
    college_index = college_player_series.index
    new_index = []
    for index in college_index:
        new_index.append(index+'_COLLEGE')
    college_player_series.index = new_index
    return college_player_series

In [8]:
def combine_nba_college(college_series,nba_series):
    return pd.concat([college_series,nba_series])

In [9]:
data = pd.DataFrame()
count = 0
for playr in lst1:
    count += 1
    print(count)
    player = playr[5]
    college_id = duplicate_college_name(playr)
    nba_id = playr[-1]
    print(player)
    print(college_id)
    try:
        college_dataframe = Player(college_id).dataframe
        college_series = college_dataframe.iloc[-1,:]
        nba_series = NBA.Player(nba_id).dataframe.iloc[0,:]
    except:
        continue
    college_series['conference'] = college_dataframe.iloc[-2,:]['conference']
    college_series['years_in_college'] = len(college_dataframe.index) - 1
    college_series['nba_position'] = playr[7]
    college_series.drop(labels=['player_id'])
    nba_series = rename_nba_index(nba_series)
    college_series = rename_college_index(college_series)
    combined = combine_nba_college(college_series,nba_series)
    data = pd.concat([data, pd.DataFrame(combined.values.reshape(1,len(combined.values)),
                                         index=[player],
                                         columns=[combined.index])])
data

1
Dragan Tarlać
not_college_player
2
Derrick Williams
derrick-williams-2
3
Mike Brittain
mike-brittain-1
4
Eric Leckner
eric-leckner-1
5
Brandon Bass
brandon-bass-1
6
Lester Hudson
lester-hudson-1
7
Nik Stauskas
nik-stauskas-1
8
Michael Dickerson
michael-dickerson-1
9
Latrell Sprewell
latrell-sprewell-1
10
Trevor Ariza
trevor-ariza-1
11
Ben Bentil
ben-bentil-1
12
Dexter Pittman
dexter-pittman-1
13
Vonteego Cummings
vonteego-cummings-1
14
Gary Leonard
gary-leonard-1
15
Dario Šarić
not_college_player
16
Rudy Gay
rudy-gay-1
17
Corey Gaines
corey-gaines-1
18
Yaroslav Korolev
not_college_player
19
Antonio Burks
antonio-burks-1
20
Ansu Sesay
ansu-sesay-1
21
Ömer Aşık
not_college_player
22
Gerald Green
not_college_player
23
Stanley Johnson
stanley-johnson-1
24
Priest Lauderdale
not_college_player
25
Nikita Wilson
nikita-wilson-1
26
Rodrigue Beaubois
not_college_player
27
Michael Jackson
michael-jackson-1
28
Monta Ellis
not_college_player
29
Von Wafer
von-wafer-1
30
Pascal Siakam
pascal-siakam

248
Damien Inglis
not_college_player
249
Tyler Harvey
tyler-harvey-1
250
Mike Peplowski
mike-peplowski-1
251
Dave Hoppen
dave-hoppen-1
252
Toney Douglas
toney-douglas-1
253
Kelly Oubre
kelly-oubre-1
254
Glen Davis
glen-davis-1
255
Raef LaFrentz
raef-lafrentz-1
256
Sean Marks
sean-marks-1
257
Desmond Mason
desmond-mason-1
258
Shareef Abdur-Rahim
shareef-abdur-rahim-1
259
Frank Kaminsky
frank-kaminsky-1
260
Bill Martin
bill-martin-1
261
Semaj Christon
semaj-christon-1
262
Yao Ming
not_college_player
263
Clifford Rozier
clifford-rozier-1
264
Rodney Monroe
rodney-monroe-1
265
Reggie Williams
reggie-williams-1
266
Quentin Richardson
quentin-richardson-1
267
Maurice Harkless
maurice-harkless-1
268
Robert Archibald
robert-archibald-1
269
Sir'Dominic Pointer
sirdominic-pointer-1
270
Željko Rebrača
not_college_player
271
Joel Embiid
joel-embiid-1
272
Al Jefferson
not_college_player
273
Sergei Karaulov
not_college_player
274
John Celestand
john-celestand-1
275
Ernest Brown
not_college_player
276

486
Daniel Orton
daniel-orton-1
487
Paul George
paul-george-1
488
Bernard Robinson
bernard-robinson-1
489
Eddie Jones
eddie-jones-1
490
Doug Christie
doug-christie-1
491
John Jenkins
john-jenkins-1
492
DeSagana Diop
not_college_player
493
Domantas Sabonis
domantas-sabonis-1
494
Marreese Speights
marreese-speights-1
495
Bryon Russell
bryon-russell-1
496
Ervin Johnson
ervin-johnson-1
497
Branden Dawson
branden-dawson-1
498
Martell Webster
not_college_player
499
Paul Millsap
paul-millsap-1
500
Kris Dunn
kris-dunn-1
501
Raul Neto
not_college_player
502
Álex Abrines
not_college_player
503
Fennis Dembo
fennis-dembo-1
504
Tony Bennett
tony-bennett-1
505
Cuttino Mobley
cuttino-mobley-1
506
Oleksiy Pecherov
not_college_player
507
Charlie Villanueva
charlie-villanueva-1
508
Dirk Nowitzki
not_college_player
509
William Bedford
william-bedford-1
510
Shabazz Napier
shabazz-napier-1
511
John Morton
john-morton-1
512
Juan Carlos Navarro
not_college_player
513
Luis Flores
luis-flores-1
514
Dexter Shou

726
Morris Almond
morris-almond-1
727
J.J. Redick
jj-redick-1
728
Dave Henderson
dave-henderson-1
729
Jason Maxiell
jason-maxiell-1
730
Alex Oriakhi
alex-oriakhi-1
731
Kyle Lowry
kyle-lowry-1
732
Gheorghe Mureșan
not_college_player
733
Devin Booker
devin-booker-2
734
Perry Jones
perry-jones-1
735
Mark Bryant
mark-bryant-1
736
Bart Kofoed
not_college_player
737
Patrick O'Bryant
patrick-obryant-1
738
Damon Stoudamire
damon-stoudamire-1
739
Voise Winters
voise-winters-1
740
Clifford Robinson
clifford-robinson-1
741
Myles Turner
myles-turner-1
742
Dean Garrett
dean-garrett-1
743
Earl Clark
earl-clark-1
744
Jamaal Franklin
jamaal-franklin-1
745
Trenton Hassell
trenton-hassell-1
746
Tyshawn Taylor
tyshawn-taylor-1
747
Jahlil Okafor
jahlil-okafor-1
748
Jerome Williams
jerome-williams-1
749
Draymond Green
draymond-green-1
750
Sergiy Gladyr
not_college_player
751
Dorell Wright
not_college_player
752
Arvydas Sabonis
not_college_player
753
Tyler Ennis
tyler-ennis-1
754
Jan Veselý
not_college_play

963
Sofoklis Schortsanitis
not_college_player
964
Derrick Rose
derrick-rose-1
965
Josh Richardson
josh-richardson-2
966
Mark Madsen
mark-madsen-1
967
Mike Taylor
mike-taylor-1
968
Kawhi Leonard
kawhi-leonard-1
969
Nicolas Batum
not_college_player
970
Myron Jackson
myron-jackson-1
971
Cal Bowdler
cal-bowdler-1
972
Joel Bolomboy
joel-bolomboy-1
973
Bryant Stith
bryant-stith-1
974
Doug Roth
doug-roth-1
975
Casey Shaw
casey-shaw-1
976
Greg Anthony
greg-anthony-1
977
Dan Dickau
dan-dickau-1
978
Kevin Martin
kevin-martin-1
979
Eric Gordon
eric-gordon-1
980
Jackson Vroman
jackson-vroman-1
981
Caron Butler
caron-butler-1
982
Andrew Nicholson
andrew-nicholson-1
983
Myron Brown
not_college_player
984
Jermaine O'Neal
not_college_player
985
Walt Williams
walt-williams-1
986
Ruben Boumtje-Boumtje
ruben-boumtje-boumtje-1
987
Drew Gooden
drew-gooden-1
988
Lou Williams
not_college_player
989
Julius Randle
julius-randle-1
990
Jerome Allen
jerome-allen-1
991
Damion James
damion-james-1
992
Kristaps Porz

1199
Dave Johnson
dave-johnson-1
1200
Orien Greene
orien-greene-1
1201
Patrick McCaw
patrick-mccaw-1
1202
Mark Davis
mark-davis-1
1203
Terry Catledge
terry-catledge-1
1204
Taj Gibson
taj-gibson-1
1205
Marquis Teague
marquis-teague-1
1206
Kendrick Perkins
not_college_player
1207
Chris Singleton
chris-singleton-1
1208
Aaron White
aaron-white-3
1209
Darren Collison
darren-collison-1
1210
Greg Dreiling
greg-dreiling-1
1211
Randy Foye
randy-foye-1
1212
Sonny Weems
sonny-weems-1
1213
Charles Smith
charles-smith-1
1214
DeMarco Johnson
demarco-johnson-1
1215
Kobe Bryant
not_college_player
1216
Erick Barkley
erick-barkley-1
1217
Andre Emmett
andre-emmett-1
1218
Mark Pope
mark-pope-1
1219
Norman Powell
norman-powell-1
1220
Dave Popson
dave-popson-1
1221
Darius Johnson-Odom
darius-johnson-odom-1
1222
Matt Barnes
matt-barnes-1
1223
Nikola Jokić
not_college_player
1224
Ronald Murray
not_college_player
1225
Michael Doleac
michael-doleac-1
1226
Travis Best
travis-best-1
1227
Tony White
tony-white-1
1

1433
Joe Arlauckas
joe-arlauckas-1
1434
Darko Miličić
not_college_player
1435
Derrick Byars
derrick-byars-1
1436
Carrick Felix
carrick-felix-1
1437
Tamar Slay
tamar-slay-1
1438
Larry Nance
larry-nance-2
1439
Kendall Gill
kendall-gill-1
1440
Charles O'Bannon
charles-obannon-1
1441
Buck Johnson
buck-johnson-1
1442
Jake Layman
jake-layman-1
1443
Billy Thompson
billy-thompson-1
1444
Ed Pinckney
ed-pinckney-1
1445
Thon Maker
not_college_player
1446
Dwayne McClain
dwayne-mcclain-1
1447
Pervis Ellison
pervis-ellison-1
1448
Brian Evans
brian-evans-1
1449
Olden Polynice
olden-polynice-1
1450
Ronnie Murphy
ronnie-murphy-1
1451
Georgi Glouchkov
not_college_player
1452
Primož Brezec
not_college_player
1453
Dwayne Schintzius
dwayne-schintzius-1
1454
Marko Milič
not_college_player
1455
Travis Leslie
travis-leslie-1
1456
Corliss Williamson
corliss-williamson-1
1457
Michael Anderson
michael-anderson-1
1458
Wayne Simien
wayne-simien-1
1459
Alec Kessler
alec-kessler-1
1460
Doug Smith
doug-smith-1
1461
D

DaJuan Summers
dajuan-summers-1
1665
Duane Cooper
duane-cooper-1
1666
Brad Newley
not_college_player
1667
Rickie Winslow
rickie-winslow-1
1668
Clarence Weatherspoon
clarence-weatherspoon-1
1669
Ryan Richards
not_college_player
1670
Donnie Boyce
donnie-boyce-1
1671
Wang Zhizhi
not_college_player
1672
Žarko Čabarkapa
not_college_player
1673
Jeff Teague
jeff-teague-1
1674
Amir Johnson
not_college_player
1675
Gani Lawal
gani-lawal-1
1676
Joe Johnson
joe-johnson-1
1677
Fred Hoiberg
fred-hoiberg-1
1678
Lucious Harris
lucious-harris-1
1679
Nedžad Sinanović
not_college_player
1680
John Salley
john-salley-1
1681
Greg Grant
not_college_player
1682
Stephon Marbury
stephon-marbury-1
1683
Morlon Wiley
morlon-wiley-1
1684
James Posey
james-posey-1
1685
Solomon Jones
solomon-jones-1
1686
A.J. Guyton
aj-guyton-1
1687
Tyrone Hill
tyrone-hill-1
1688
Robert Traylor
robert-traylor-1
1689
Jonas Valančiūnas
not_college_player
1690
Brandon Jennings
not_college_player
1691
Cory Alexander
cory-alexander-1
1692

,assist_percentage_COLLEGE,assists_COLLEGE,block_percentage_COLLEGE,blocks_COLLEGE,box_plus_minus_COLLEGE,conference_COLLEGE,defensive_box_plus_minus_COLLEGE,defensive_rebound_percentage_COLLEGE,defensive_rebounds_COLLEGE,defensive_win_shares_COLLEGE,...,turnovers_NBA,two_point_attempts_NBA,two_point_percentage_NBA,two_pointers_NBA,two_pointers_assisted_percentage_NBA,usage_percentage_NBA,value_over_replacement_player_NBA,weight_NBA,win_shares_NBA,win_shares_per_48_minutes_NBA
Derrick Williams,7.7,65,2.3,46,11.8,pac-10,2.6,21,360,3,...,77,360,0.467,168,0.613,20.7,-0.4,240,1.8,0.059
Mike Brittain,None,90,None,160,None,metro,None,None,None,None,...,20,43,0.512,22,None,13.1,-0.3,235,0,-0.01
Eric Leckner,None,54,None,164,None,wac,None,None,None,None,...,69,220,0.545,120,None,18.4,-0.5,265,1.5,0.093
Brandon Bass,5.5,42,3.3,104,None,sec,None,None,333,3.9,...,11,70,0.4,28,0.607,15.6,-0.4,250,0.1,0.013
Lester Hudson,28,283,1.2,40,None,ovc,None,None,371,5.4,...,14,37,0.432,16,0.5,25,0,190,0.1,0.029
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Devin Harris,28.8,295,0.3,40,None,big-ten,None,None,328,7.8,...,82,235,0.481,113,0.257,18.8,0.9,185,2.7,0.111
Harold Keeling,None,244,None,49,None,wcac,None,None,None,None,...,7,39,0.436,17,None,28.4,0,185,0.1,0.041
Otis Smith,None,272,None,211,None,sun-belt,None,None,None,None,...,19,77,0.429,33,None,24.5,-0.2,210,-0.2,-0.053
Cedric Henderson,None,25,None,31,None,sec,None,None,None,None,...,4,8,0.5,4,None,34.2,-0.1,210,0,-0.015


In [10]:
data.columns = [col[0] for col in data.columns]

In [11]:
data.to_pickle('data2.pkl')

In [12]:
#data = pd.read_pickle('data2.pkl')

In [13]:
data.drop(columns =['height_NBA','player_id_COLLEGE','player_id_NBA','nationality_NBA','position_NBA','team_abbreviation_NBA'], inplace=True)


In [14]:
data = pd.get_dummies(data=data,columns=['position_COLLEGE','nba_position_COLLEGE','conference_COLLEGE','team_abbreviation_COLLEGE']) 

In [15]:
def height_func(row):
    try: 
        split = row['height_COLLEGE'].split('-')
    except:
        return -1
    return int(split[0])+(int(split[1])/12)

In [16]:
list(data.columns)

['assist_percentage_COLLEGE',
 'assists_COLLEGE',
 'block_percentage_COLLEGE',
 'blocks_COLLEGE',
 'box_plus_minus_COLLEGE',
 'defensive_box_plus_minus_COLLEGE',
 'defensive_rebound_percentage_COLLEGE',
 'defensive_rebounds_COLLEGE',
 'defensive_win_shares_COLLEGE',
 'effective_field_goal_percentage_COLLEGE',
 'field_goal_attempts_COLLEGE',
 'field_goal_percentage_COLLEGE',
 'field_goals_COLLEGE',
 'free_throw_attempt_rate_COLLEGE',
 'free_throw_attempts_COLLEGE',
 'free_throw_percentage_COLLEGE',
 'free_throws_COLLEGE',
 'games_played_COLLEGE',
 'games_started_COLLEGE',
 'height_COLLEGE',
 'minutes_played_COLLEGE',
 'offensive_box_plus_minus_COLLEGE',
 'offensive_rebound_percentage_COLLEGE',
 'offensive_rebounds_COLLEGE',
 'offensive_win_shares_COLLEGE',
 'personal_fouls_COLLEGE',
 'player_efficiency_rating_COLLEGE',
 'points_COLLEGE',
 'points_produced_COLLEGE',
 'steal_percentage_COLLEGE',
 'steals_COLLEGE',
 'three_point_attempt_rate_COLLEGE',
 'three_point_attempts_COLLEGE',
 'thr

In [17]:
data['height_COLLEGE'] = data.apply(height_func ,axis=1)

In [18]:
for column in data.columns:
    print(column)
    data[column].astype(float)
    length_of_nulls=len(data[data[column].isnull()])
    if length_of_nulls > 0:
        data[column +' is missing'] = data[column].isnull()*1
data.fillna(-1, inplace=True)

assist_percentage_COLLEGE
assists_COLLEGE
block_percentage_COLLEGE
blocks_COLLEGE
box_plus_minus_COLLEGE
defensive_box_plus_minus_COLLEGE
defensive_rebound_percentage_COLLEGE
defensive_rebounds_COLLEGE
defensive_win_shares_COLLEGE
effective_field_goal_percentage_COLLEGE
field_goal_attempts_COLLEGE
field_goal_percentage_COLLEGE
field_goals_COLLEGE
free_throw_attempt_rate_COLLEGE
free_throw_attempts_COLLEGE
free_throw_percentage_COLLEGE
free_throws_COLLEGE
games_played_COLLEGE
games_started_COLLEGE
height_COLLEGE
minutes_played_COLLEGE
offensive_box_plus_minus_COLLEGE
offensive_rebound_percentage_COLLEGE
offensive_rebounds_COLLEGE
offensive_win_shares_COLLEGE
personal_fouls_COLLEGE
player_efficiency_rating_COLLEGE
points_COLLEGE
points_produced_COLLEGE
steal_percentage_COLLEGE
steals_COLLEGE
three_point_attempt_rate_COLLEGE
three_point_attempts_COLLEGE
three_point_percentage_COLLEGE
three_pointers_COLLEGE
total_rebound_percentage_COLLEGE
total_rebounds_COLLEGE
true_shooting_percentage_CO

In [19]:
print(len(data.index))
train_index = int(len(data.index)// (20/15))
test_index = (len(data.index)-train_index)
print(train_index)
print(test_index)

1368
1026
342


In [20]:
train_data = data.iloc[0:train_index,:]
test_data = data.iloc[train_index+1:-1,:]

In [21]:
nba = []
for player in train_data.index:
    if player in all_star_list:
        nba.append(1)
    else: 
        nba.append(0)
nba2 = []
for player in test_data.index:
    if player in all_star_list:
        nba2.append(1)
    else: 
        nba2.append(0)

In [22]:
grad_boost = GradientBoostingClassifier(learning_rate=.01, n_estimators=5000)

In [23]:
grad_boost.fit(train_data, nba)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.01, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=5000,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

In [24]:
yhat = grad_boost.predict(test_data)
tn, fp, fn, tp = confusion_matrix(nba2,yhat).ravel()
print((tn, fp, fn, tp))
print(test_data.index[((np.array(yhat)==1) & (np.array(nba2)==1))])
print(roc_auc_score(nba2,yhat))
print(test_data.index[((np.array(yhat)==1) & (np.array(nba2)==0))])
print(test_data.index[((np.array(yhat)==0) & (np.array(nba2)==1))])

(283, 8, 31, 18)
Index(['Anthony Davis', 'Chris Paul', 'Paul Pierce', 'James Harden',
       'Stephen Curry', 'Brad Daugherty', 'Carlos Boozer', 'Anfernee Hardaway',
       'Dikembe Mutombo', 'Larry Johnson', 'Dan Majerle', 'Karl-Anthony Towns',
       'Andre Drummond', 'Christian Laettner', 'Stephon Marbury',
       'Kevin Durant', 'Jerry Stackhouse', 'Brandon Roy'],
      dtype='object')
0.66992776492
Index(['Nick Vanos', 'Andrew Bogut', 'Michael Carter-Williams',
       'John Williams', 'John Salley', 'James Posey', 'Willie Anderson',
       'Harold Keeling'],
      dtype='object')
Index(['Rik Smits', 'Mookie Blaylock', 'Jamaal Magloire', 'Roy Hibbert',
       'Nikola Vučević', 'B.J. Armstrong', 'Kenny Anderson', 'Gilbert Arenas',
       'Kevin Love', 'Sam Cassell', 'David Lee', 'Jrue Holiday', 'Larry Nance',
       'Damian Lillard', 'D'Angelo Russell', 'Dana Barros', 'Rajon Rondo',
       'David West', 'Glenn Robinson', 'Gordon Hayward', 'Allan Houston',
       'Danny Manning', 'Mi

In [219]:
probs = grad_boost.predict_proba(test_data)
prob_list = []
for element in probs:
    prob_list.append(element[1])
prob_array = np.array(prob_list)
index_array = np.array(test_data.index)
print(index_array[prob_array>.2])
#grad_boost.predict(data.loc['Frank Kaminsky'])

['Brendan Haywood' 'Damon Stoudamire' 'Andrew Wiggins' 'Vince Carter'
 'Chris Mullin' 'Frank Kaminsky' 'Ron Harper' 'Domantas Sabonis'
 'Patrick Ewing' 'Brandon Roy' 'Dennis Hopson' 'Karl-Anthony Towns'
 'Patrick Ewing' 'Delon Wright' 'Jason Kidd' 'James Posey' 'Armon Johnson'
 'Chris Webber' 'Rodney Stuckey' 'Jim Jackson' 'Khalid Reeves'
 'Oliver Miller' 'Derrick Rose' 'John Wall' 'Dwyane Wade']


In [191]:
random_forest=RandomForestClassifier()

In [193]:
random_forest.fit(train_data, nba)
yhat = random_forest.predict(test_data)
tn, fp, fn, tp = confusion_matrix(nba2,yhat).ravel()
print((tn, fp, fn, tp))
print(test_data.index[((np.array(yhat)==1) & (np.array(nba2)==1))])
print(roc_auc_score(nba2,yhat))
print(test_data.index[((np.array(yhat)==1) & (np.array(nba2)==0))])
print(test_data.index[((np.array(yhat)==0) & (np.array(nba2)==1))])

(298, 2, 28, 3)
Index(['Patrick Ewing', 'Jason Kidd', 'Chris Webber'], dtype='object')
0.545053763441
Index(['Ron Harper', 'James Posey'], dtype='object')
Index(['Al Horford', 'Anthony Mason', 'Vince Carter', 'Steve Smith',
       'Chris Mullin', 'Kenny Anderson', 'Kawhi Leonard', 'Paul George',
       'Brandon Roy', 'Isaiah Thomas', 'Chris Kaman', 'Deron Williams',
       'Karl-Anthony Towns', 'Patrick Ewing', 'Antawn Jamison',
       'Michael Adams', 'Andre Drummond', 'Jeff Hornacek', 'Dana Barros',
       'Derrick Rose', 'John Wall', 'Devin Harris', 'Brook Lopez',
       'Joe Dumars', 'Mookie Blaylock', 'Luol Deng', 'Carlos Boozer',
       'Dwyane Wade'],
      dtype='object')


In [264]:
def height_func(row):
    try: 
        split = row['height'].split('-')
    except:
        return 6.56
    return int(split[0])+(int(split[1])/12)